In [1]:
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from flask import Flask, jsonify

# Database Setup
#################################################
engine = create_engine("sqlite:///Resources/hawaii.sqlite",echo=False)

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)


# Save reference to the table
Measurement = Base.classes.measurement
Station = Base.classes.station

#################################################
# Flask Setup
#################################################
app = Flask(__name__)


#################################################
# Flask Routes
#################################################

@app.route('/')
def homepage():
    '''List all available routes.'''
    return(
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/<start><br/>"
        f"/api/v1.0/<start>/<end><br/>"
    )    

@app.route("/api/v1.0/precipitation")
def precipitation():
    # Create our session (link) from Python to the DB
    
    session = Session(engine)

    # Query all date and precipitation
    
    results = session.query(Measurement.prcp).all()
    dates = session.query(Measurement.date).all()
    
    #Convert the query results to a dictionary using `date` as the key and `prcp` as the value.
    
    dic={dates[i][0]:results[i][0] for i in range(0,len(results))}

    session.close()

    # Return the JSON representation of your dictionary.

    return jsonify(dic)

In [5]:
# @app.route("/api/v1.0/stations")
# def stations():
    
    # Create our session (link) from Python to the DB
    
session = Session(engine)
 
    # Query all stations
    
stations = session.query(Station.station).all()
stations=[stations[i][0] for i in range(0,len(stations))]
stations
# session.close()

    # Return a JSON list of stations from the dataset
    
# return jsonify(stations)

['USC00519397',
 'USC00513117',
 'USC00514830',
 'USC00517948',
 'USC00518838',
 'USC00519523',
 'USC00519281',
 'USC00511918',
 'USC00516128']

In [3]:
@app.route("/api/v1.0/tobs")
def tobs():
    
    # Create our session (link) from Python to the DB
    
    session = Session(engine)
 
    # Query the most active station.
    
    counts=func.count(Measurement.station)
    
    Mostactivity=session.query(Measurement.station).group_by(Measurement.station).order_by(counts.desc()).first()
    
    most_active_station=Mostactivity[0]

    # Query the dates the most active station for the previous year of data
    
    import datetime as dt
    from datetime import date, timedelta, datetime

    most_recent_date=session.query(Measurement.date).filter(Measurement.station==most_active_station).\
        order_by(Measurement.date.desc()).first()
    
    most_recent_date=dt.datetime.strptime(most_recent_date[0], '%Y-%m-%d').date()
    
    DateYaerAgo=most_recent_date-timedelta(days=365)
    
    # Query the most active station for the previous year of data
    
    Post12M_most_active_station=session.query(Measurement.tobs).\
        filter(Measurement.date>=DateYaerAgo).filter(Measurement.station==most_active_station).all()

    results=[Post12M_most_active_station[i][0] for i in range(0,len(Post12M_most_active_station))]
    
    session.close()

    # Return a JSON list of temperature observations (TOBS) for the previous year.
    
    return jsonify(results)

@app.route("/api/v1.0/<start>")

def DataAfterStartDate(start):
    """Fetch the data after start date which matchs the path variable supplied by the user, or a 404 if not."""
    import re
    
    # Create our session (link) from Python to the DB
    
    session = Session(engine)
 
    # Query 
     
    canonicalised = re.sub("([^\u0030-\u0039])", '', start)
    
    for date in Measurement:
        search_term = Measurement['date'][0].replace('-','')
        
        if search_term == canonicalised:
            
            StartDate=dt.datetime.strptime(Measurement['date'][0], '%Y-%m-%d').date()
            TMIN=session.query(func.min(Measurement.tobs)).filter(Measurement.date>=StartDate)
            TAVG=session.query(func.avg(Measurement.tobs)).filter(Measurement.date>=StartDate)
            TMAX=session.query(func.max(Measurement.tobs)).filter(Measurement.date>=StartDate)
            return jsonify(f"The minimun temperature after {start} is {TMIN}."\
                           f"The average temperature after {start} is {TAVG}."\
                           f"The maximum temperature after {start} is {TMAX}.")
    

    session.close()

    # Return a JSON list of stations from the dataset
    
    return jsonify({"error": f"Data with {start} not found."}), 404

# @app.route('/api/v1.0/<start>/<end>')
# def stations():
    
#     # Create our session (link) from Python to the DB
    
#     session = Session(engine)
 
#     # Query all stations
    
#     stations = session.query(Station.station).all()

#     session.close()

#     # Return a JSON list of stations from the dataset
    
#     return jsonify(stations)



if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use